In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib

from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [41]:
NUMBER_TORNADOS = 111 #Number of Tornado Events
NUMBER_RANDOM = 110 #Number of Random Events
NUMBER_DAYS = 14 #Number of days of data per Tornado/Event (Max: 57)
FILE_NAME = "historical_data_2.csv" #CSV file that contains the data
NUMBER_OF_ROWS = 296

In [42]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', NUMBER_OF_ROWS)
pd.set_option('display.min_rows', 20)

In [3]:
#Field 0: 'datatime'
#Field 1: 'temperature'
#Field 2: 'windspeed'
#Field 3: 'surface solar radiation' -Alvaro
#Field 4: 'relative humidity' -Abdullah
#Field 5: 'surface pressure' -Frazier
#Field 6: 'total precipitation' -Simon
#Field 7: 'city'
#Field 8: 'event_id'
#Field 9: 'latitude'
#Field 10: 'longitude'
#Field 11: 'outcome'

df = pd.read_csv(FILE_NAME)
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [4]:
df 

datetime  temperature (degC)  wind_speed (m/s)  \
0      2009-11-25                9.91              2.76   
1      2009-11-26                9.35              2.11   
2      2009-11-27               13.11              3.30   
3      2009-11-28               16.85              3.27   
4      2009-11-29               12.53              3.04   
...           ...                 ...               ...   
16867  2017-02-24                4.32              5.83   
16868  2017-02-25                2.52              1.58   
16869  2017-02-26                9.50              2.33   
16870  2017-02-27               15.56              4.96   
16871  2017-02-28               20.36              6.07   

       surface_solar_radiation (W/m^2)  relative_humidity (0-1)  \
0                                  160                     0.47   
1                                  145                     0.60   
2                                  111                     0.63   
3                                   71                     0.83   
4                                   98                     0.79   
...                                ...                      ...   
16867                              187                     0.49   
16868                              155                     0.46   
16869                              183                     0.61   
16870                               54                     0.77   
16871                               18                     0.82   

       surface_pressure (Pa)  total_precipitation (mm of water equivalent)  \
0                  101257.58                                          0.00   
1                  101181.89                                          0.00   
2                  100649.88                                          0.00   
3                  100267.40                                          0.26   
4                  100664.11                                          0.92   
...                      ...                                           ...   
16867               99717.88                                          0.00   
16868              100450.42                                          0.00   
16869              100341.69                                          0.04   
16870               99730.39                                          0.01   
16871               99075.00                                          0.00   

       latitude  longitude  event_id  
0       32.4869   -94.1689    203448  
1       32.4869   -94.1689    203448  
2       32.4869   -94.1689    203448  
3       32.4869   -94.1689    203448  
4       32.4869   -94.1689    203448  
...         ...        ...       ...  
16867   37.9392   -89.1503    677492  
16868   37.9392   -89.1503    677492  
16869   37.9392   -89.1503    677492  
16870   37.9392   -89.1503    677492  
16871   37.9392   -89.1503    677492  

[16872 rows x 10 columns]

In [5]:
#df = data.drop(columns=['Unnamed: 0'])
# Get a seto of the event Id's
event_ids = set(df['event_id'].to_numpy())

In [6]:
## Produces a list of dfs
# Each df has the rolling means of the fundamental features
list_of_dfs = []
#Use IDs to loop over events 
for i in event_ids:
    # We initialize a df per event
    event_df = pd.DataFrame()
    # Get data for that event 
    event = df.loc[df['event_id'] == i]
    # Save the dates
    temp = event['datetime']
    # We only want the features we are processing
    fundamental_features = event.drop(columns=['datetime', 'latitude', 'longitude', 'event_id'	], axis = 1)
    # Compute the rolling mean
    event_df = fundamental_features.expanding().mean() 
    # Put the date
    event_df['Date'] = temp 
    # We want 3 weeks of data
    event_df = event_df.tail(21)
    # Add to list
    list_of_dfs.append(event_df)

In [7]:
len(list_of_dfs)
len(list_of_dfs[0])

21

In [8]:
list_of_dfs[0]

temperature (degC)  wind_speed (m/s)  surface_solar_radiation (W/m^2)  \
16566           15.615946          3.614595                        96.783784   
16567           15.732105          3.584737                        96.842105   
16568           15.856923          3.569231                        96.102564   
16569           15.942250          3.595000                        97.225000   
16570           15.934634          3.692439                        99.243902   
16571           15.898571          3.608333                       101.214286   
16572           15.948140          3.616744                       101.790698   
16573           15.949773          3.631591                       103.000000   
16574           15.820889          3.652889                       104.644444   
16575           15.696087          3.633478                       104.326087   
16576           15.607234          3.666596                       106.191489   
16577           15.556042          3.648958                       107.979167   
16578           15.562449          3.665306                       109.612245   
16579           15.587800          3.670400                       110.240000   
16580           15.621765          3.636078                       110.960784   
16581           15.619808          3.599808                       110.288462   
16582           15.528302          3.601321                       111.471698   
16583           15.521481          3.580926                       112.666667   
16584           15.568364          3.574364                       113.000000   
16585           15.633929          3.591607                       111.839286   
16586           15.744035          3.640175                       110.052632   

       relative_humidity (0-1)  surface_pressure (Pa)  \
16566                 0.821351          102196.950811   
16567                 0.823684          102163.660789   
16568                 0.825385          102122.925385   
16569                 0.823500          102066.205500   
16570                 0.818537          102029.591707   
16571                 0.815952          102011.347143   
16572                 0.815581          101990.168140   
16573                 0.810455          101981.880000   
16574                 0.803556          101989.729556   
16575                 0.796304          101997.687174   
16576                 0.789574          102000.070213   
16577                 0.784375          102005.124792   
16578                 0.782245          102007.550000   
16579                 0.781000          102006.680800   
16580                 0.781373          102008.689020   
16581                 0.782692          102014.534423   
16582                 0.778679          102026.985283   
16583                 0.777963          102031.097037   
16584                 0.779273          102030.379091   
16585                 0.781429          102017.196250   
16586                 0.782456          101995.665965   

       total_precipitation (mm of water equivalent)        Date  
16566                                      0.167027  2017-01-18  
16567                                      0.164737  2017-01-19  
16568                                      0.197436  2017-01-20  
16569                                      0.198750  2017-01-21  
16570                                      0.194390  2017-01-22  
16571                                      0.189762  2017-01-23  
16572                                      0.185581  2017-01-24  
16573                                      0.182273  2017-01-25  
16574                                      0.178222  2017-01-26  
16575                                      0.174348  2017-01-27  
16576                                      0.170638  2017-01-28  
16577                                      0.167083  2017-01-29  
16578                                      0.163673  2017-01-30  
16579                                      0.160400  2017-01-31  
1

In [24]:
def generateRollingAvgFeatures(df):
    ''' Turns a whole data frame into a line of rolling average features
    '''
    all_lists = []
    #Iterate over ever column
    for column in df.columns[:-1]:
        # Processed list
        # Turn the column of data into a list
        processing_list = df[str(column)].tolist()
        #Loop over the list 
        # Initialize a list to store the 7 processed values 
        processed_list = []
        for i in range(len(processing_list)):
            # Gets the right spots for computing
            if (((i + 1) % 3) == 0):
                feature_point = (processing_list[i] + processing_list[i -1] + processing_list[i-2]) / 3
                processed_list.append(feature_point)
                # TODO: Should the data be normalized within its own sample at this point ?
        # So we end up w 1 list
        all_lists += processed_list
    return all_lists 
        # TODO: We finished processing the data , we need to turn it into a row of features


In [30]:
# Loop over each Df
processing_data = list_of_dfs.copy()
flag = 0
df = pd.DataFrame()
list_of_lists = []
for df in processing_data:
    if (1):
        # This will return a row of features for each event 
        
        # TODO:Should this return a list that then becomes a row in the dataframe?
        current = generateRollingAvgFeatures(df)
        list_of_lists.append(current)
        flag +=1
        # We need to add this df to the 


# Pass the DF
# For each DF run computations and generate columns -> Features

In [31]:
len(current)

42

In [32]:
len(list_of_lists)

296

In [37]:
nums = range(7)
cols = [f'{v}_{i}' for v in df.columns[:-1] for i in nums]

In [38]:
processed_data = pd.DataFrame(list_of_lists, columns=cols)

In [44]:
processed_data.head(20)

temperature (degC)_0  temperature (degC)_1  temperature (degC)_2  \
0              15.734991             15.925152             15.906267   
1              13.043718             13.463723             13.452982   
2               8.224683              8.853130              9.080050   
3               8.205205              8.831636              9.092504   
4               8.318497              8.945491              9.241553   
5              15.234227             15.725689             15.905757   
6              13.885902             14.255321             14.253469   
7               8.319908              8.946878              9.244729   
8              15.297785             15.799603             15.941389   
9              12.681554             12.417041             12.522798   
10             14.008977             13.925108             13.941365   
11             14.658948             14.861397             14.810140   
12             14.861600             15.233809             15.542875   
13             11.605010             11.663981             12.070632   
14             14.227937             14.452400             14.046104   
15             14.089933             14.627614             14.794241   
16             22.827193             23.058029             23.171095   
17             14.491090             14.788493             14.798965   
18             14.229118             14.790328             15.390349   
19             23.586682             23.786359             23.882306   

    temperature (degC)_3  temperature (degC)_4  temperature (degC)_5  \
0              15.619788             15.590671             15.556530   
1              13.562694             13.877710             14.252325   
2               9.079007              9.459363              9.991274   
3               9.093781              9.467477             10.024016   
4               9.242445              9.616028             10.215082   
5              15.993587             16.382112             16.778862   
6              14.328641             14.591385             14.966385   
7               9.245847              9.619294             10.220291   
8              15.995497             16.358627             16.726287   
9              12.764305             13.115110             13.458239   
10             13.956879             14.031913             14.381469   
11             14.473965             14.473258             14.439082   
12             15.762294             15.954820             16.093915   
13             11.848368             11.572450             11.492523   
14             14.012144             14.098660             13.896238   
15             14.834039             15.172704             15.554613   
16             23.459230             23.539398             23.497575   
17             15.104720             15.315076             15.640561   
18             15.769748             16.112992             16.579592   
19             23.503416             23.031756             22.752382   

    temperature (degC)_6  wind_speed (m/s)_0  wind_speed (m/s)_1  \
0              15.648776            3.589521            3.631924   
1              14.716715            2.357845            2.429818   
2              10.371327            4.213247            4.287833   
3              10.420423            4.258746            4.325136   
4              10.636644            4.296162            4.359084   
5              17.159001            2.886413            2.952579   
6              15.458381            2.281571            2.337522   
7              10.642771            4.295381            4.357784   
8              17.089253            2.898456            2.953396   
9              13.695519            3.227399            3.194605   
10             14.707025            3.263720            3.292516   
11             14.604367            2.675562            2.708469   
12             16.369417            4.048920            3.922918   
13             11.515684  

##Helper Function

In [83]:
# Takes input two column names of a dataframe and returns the ratio between then returns the ratio of them as a new figure in the dataframe inputed

def create_ratio_columns(numerator, denominator, df):
    ''' numerator, denominator are column names 
        df is the df where the columns are 
    ''' 
    # filter based on names
    f1 = df.filter(like=numerator)
    f2 = df.filter(like=denominator)
    # Init list for each column of ratio
    ratios = list()
    # Iterating through one set of column to find appropriate data
    for i, c in enumerate(f1.columns):
        col = f'{numerator}/{denominator}:{i}'
        r = f1.iloc[:, i].div(f2.iloc[:, i])
        r.name = col
        # ASS 
        ratios.append(r)
        
    
    ratios = pd.concat(ratios, axis=1)
    
    return ratios
    

In [85]:
ratio_df = create_ratio_columns('temp', 'wind', processed_data)
ratio_df.head()

temp/wind:0  temp/wind:1  temp/wind:2  temp/wind:3  temp/wind:4  \
0     4.383591     4.384770     4.377380     4.279772     4.262936   
1     5.532051     5.541042     5.371608     5.414803     5.620181   
2     1.952101     2.064710     2.039192     1.996681     2.105694   
3     1.926672     2.041932     2.044232     2.001689     2.104063   
4     1.936262     2.052150     2.069376     2.028727     2.126805   

   temp/wind:5  temp/wind:6  
0     4.328451     4.344410  
1     5.887272     5.912216  
2     2.268891     2.385568  
3     2.264027     2.381820  
4     2.280078     2.404959

In [90]:
new = pd.concat([processed_data, ratio_df], axis=1)

In [ ]:
new

In [ ]:
## Plot data
## Take any 2 columns and plot how they relate to eachother in a line 
def create_line(df_column1, df_column2):
    ''' We want to be able to visualize the behavior to explore the features more 
    '''
    
    # Find how windspeed and humidity grow together with time. 

In [ ]:
test1

## Drawing Board

In [51]:
dfb = pd.DataFrame()

In [52]:
type(dfb)

pandas.core.frame.DataFrame

In [53]:
dfb['b'] = [0, 1, 2, 3, 4]

In [54]:
dfb['c'] = [0, 1, 2, 3, 4]

In [55]:
dfb

b  c
0  0  0
1  1  1
2  2  2
3  3  3
4  4  4

In [56]:
dfb.expanding().mean()

b    c
0  0.0  0.0
1  0.5  0.5
2  1.0  1.0
3  1.5  1.5
4  2.0  2.0